# Zero-Shot Transfer Learning with CLIP

In [2]:
import torch
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
import requests

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

Using device cpu
